<a href="https://colab.research.google.com/github/JosephGeorgeMtech/UNET_nucleiDataset/blob/main/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
tf.random.set_seed(42)

import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt
%matplotlib inline

import glob
import PIL
from PIL import Image
import os
import cv2

In [ ]:
! unzip '/content/drive/MyDrive/nucleiData.zip' -d '/content/nucleiDataset/'

Training data

In [ ]:
trainFiles = glob.glob("/content/nucleiDataset/nucleiData/nTrain/*_gray.pgm")

print(trainFiles[:10])

X_train = []
y_train = []

for filePath in trainFiles:
    X_i = Image.open(filePath)
    X_i = np.array(X_i)/255.0
    
    X_train.append(X_i)
    
    y_i = Image.open(filePath.replace("gray", "mask"))
    y_i = np.array(y_i)/255
    
    y_train.append(y_i)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)

print(X_train.shape)
print(y_train.shape)

In [ ]:
plt.imshow(X_train[0], cmap='gray')
plt.show()

In [ ]:
plt.imshow(y_train[0], cmap='gray')
plt.show()

Test data

In [ ]:
testFiles = glob.glob("/content/nucleiDataset/nucleiData/nTest/*_gray.pgm")

print(testFiles[:10])

X_test = []
y_test = []

for filePath in testFiles:
    X_i = Image.open(filePath)
    X_i = np.array(X_i)/255.0
    
    X_test.append(X_i)
    
    y_i = Image.open(filePath.replace("gray", "mask"))
    y_i = np.array(y_i)/255
    
    y_test.append(y_i)

In [ ]:
X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_test.shape)
print(y_test.shape)

In [ ]:
plt.imshow(X_test[1], cmap='gray')
plt.show()

In [ ]:
plt.imshow(y_test[1], cmap='gray')
plt.show()


The model requires the input to be 4D

np.newaxis is to add new dimension to the array. The position of the np.newaxis represents where I want to add dimensions.

a([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]) : shape is (10,) \

a[:, np.newaxis] : shape is (10,1)\

a[np.newaxis, :] : shape is (1,10)

numpy.repeat(a, repeats, axis=None) # Repeat elements of an array.


In [ ]:
X_train = X_train[..., np.newaxis]
y_train = y_train[..., np.newaxis]


print(X_train.shape)
print(y_train.shape)


X_test = X_test[..., np.newaxis]
y_test = y_test[..., np.newaxis]


print(X_test.shape)
print(y_test.shape)

X_train=np.repeat(X_train,3,-1) #-1 indicates the last dimension, 
                                              #which indicates the axis along whic to repeat values
X_test=np.repeat(X_test,3,-1)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

'''
X_train=np.repeat(X_train[..., np.newaxis],3,-1) #-1 indicates the last dimension, 
                                              #which indicates the axis along whic to repeat values
y_train= y_train[..., np.newaxis]
#y_train=np.repeat(y_train[..., np.newaxis],1,-1)
print(X_train.shape)
print(y_train.shape)
'''
'''
X_test = X_test[..., np.newaxis]
y_test = y_test[..., np.newaxis]
print(X_test.shape)
print(y_test.shape)
'''
'''
X_test=np.repeat(X_test[..., np.newaxis],3,-1)
y_test=np.repeat(y_test[..., np.newaxis],1,-1)
print(X_test.shape)
print(y_test.shape)
'''

In [ ]:
'''
X_train = X_train[..., np.newaxis]
y_train = y_train[..., np.newaxis]


print(X_train.shape)
print(y_train.shape)
'''
'''
X_train=np.repeat(X_train[..., np.newaxis],3,-1) #-1 indicates the last dimension, 
                                              #which indicates the axis along whic to repeat values
y_train= y_train[..., np.newaxis]
#y_train=np.repeat(y_train[..., np.newaxis],1,-1)
print(X_train.shape)
print(y_train.shape)
'''
'''
X_test = X_test[..., np.newaxis]
y_test = y_test[..., np.newaxis]
print(X_test.shape)
print(y_test.shape)
'''

X_test=np.repeat(X_test[..., np.newaxis],3,-1)
y_test=y_test[..., np.newaxis]
print(X_test.shape)
print(y_test.shape)

New attempt

In [ ]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

In [ ]:
#https://keras.io/api/applications/

In [ ]:
from keras import applications

vgg_model = applications.VGG16(weights='imagenet',
                               include_top=False,
                               input_shape=(128, 128, 3))


In [ ]:
# Creating dictionary that maps layer names to the layers
layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])

In [ ]:
layer_dict

In [ ]:
l1 = layer_dict['block1_conv2'].output
l2 = layer_dict['block2_conv2'].output
l3 = layer_dict['block3_conv3'].output
l4 = layer_dict['block4_conv3'].output
l5 = layer_dict['block5_conv3'].output
l6_pool= layer_dict['block5_pool'].output

#Upsampling to level 5
u5 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same') (l6_pool)
u5 = concatenate([u5, l5])

c5 = Conv2D(512, (3, 3), activation='relu',  padding='same') (u5)
c5 = Conv2D(512, (3, 3), activation='relu',  padding='same') (c5)
c5 = Conv2D(512, (3, 3), activation='relu',  padding='same') (c5)

#Upsampling to level 4
u4 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same') (c5)
u4 = concatenate([u4, l4])

c4 = Conv2D(512, (3, 3), activation='relu',  padding='same') (u4)
c4 = Conv2D(512, (3, 3), activation='relu',  padding='same') (c4)
c4 = Conv2D(512, (3, 3), activation='relu',  padding='same') (c4)

#Upsampling to level 3

u3 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same') (c4)
u3 = concatenate([u3, l3])

c3 = Conv2D(256, (3, 3), activation='relu',  padding='same') (u3)
c3 = Conv2D(256, (3, 3), activation='relu',  padding='same') (c3)
c3 = Conv2D(256, (3, 3), activation='relu',  padding='same') (c3)

#Upsampling to level 2
u2 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c3)
u2 = concatenate([u2, l2])

c2 = Conv2D(128, (3, 3), activation='relu',  padding='same') (u2)
c2 = Conv2D(128, (3, 3), activation='relu',  padding='same') (c2)

#Upsampling to level 1
u1 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c2)
u1 = concatenate([u1, l1])

c1 = Conv2D(64, (3, 3), activation='relu' ,padding='same') (u1)
c1 = Conv2D(64, (3, 3), activation='relu', padding='same') (c1)

#Final 1*1 convolution 
outputs = Conv2D(1, (1, 1), activation='sigmoid') (c1)

UNet_model = Model(inputs=[vgg_model.input], outputs=[outputs])

In [ ]:
UNet_model.summary()

In [ ]:
tf.keras.utils.plot_model(
    UNet_model,
    to_file='UNet_model.png',
    show_shapes=False,
    show_layer_names=False,
    rankdir='LR',
    expand_nested=False,
    dpi=96
)

In [ ]:
UNet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history_model = UNet_model.fit(X_train, y_train, validation_split=0.1, batch_size=32, epochs=50)

In [ ]:
testLoss, testAccuracy = UNet_model.evaluate(X_test, y_test)
print("Test-loss: %f, Test-accuracy: %f" % (testLoss, testAccuracy))

Predicted segmentation (grayscale)

In [ ]:
m1 = UNet_model.predict(X_test[10:11])
plt.imshow(np.squeeze(m1), cmap='gray')

Predicted segmentation (binary)

In [ ]:
prediciton_t = (m1 > 0.5).astype(np.uint8)
plt.imshow(np.squeeze(prediciton_t), cmap='gray')

Original Input

In [ ]:
plt.imshow(np.squeeze(X_test[10:11]), cmap='gray')
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)

fig.set_figheight(20)
fig.set_figwidth(20)

ax1.set_title('Original')
ax1.imshow(np.squeeze(X_test[10:11]), cmap='gray')

ax2.set_title('Segmented - grayscale')
ax2.imshow(np.squeeze(m1), cmap='gray')

ax3.set_title('Segmented - binary')
ax3.imshow(np.squeeze(prediciton_t), cmap='gray')

plt.show()